## <center>Деревья решений: классификация и регрессия</center>

### Описание задачи

Реализована модель дерева решений для задач классификации и регрессии. Для построения дерева, по аналогии с реализацией деревьев в scikit-learn, используется модифицированный [алгоритм CART](https://en.wikipedia.org/wiki/Predictive_analytics#Classification_and_regression_trees_.28CART.29).

### Описание математической основы

#### Построение деревьев решений

Для начала рассмотрим задачу классификации для объектов только с категориальными признаками и механизм построения решающих деревьев, основанный на концепции информационной энтропии (алгоритм [ID3](https://en.wikipedia.org/wiki/ID3_algorithm)).

Энтропия, интуитивно, это мера непоределенности в системе. Итак, энтропия Шеннона для системы с $K$ возможными состояниями определяется как:

$$
S = - \sum_{k=1}^{K} p_k \log_2 p_k
$$

где $p_i$ - вероятность нахождения системы в $i$-ом состоянии (фактически, доля объектов класса $i$ в выборке):

$$
p_i = \dfrac {N_i}{N}
$$

Это выражение можно трактовать как оценку среднего количества информации, необходимого для определения класса примера из выборки. Максимум энтропии достигается, когда каждое состояние системы равновероятно.

Пусть у нас имеется выборка $X$ объектов и вектор ответов $y$.

Если выполнить разбиение выборки по признаку $x_j$, то ту же оценку среднего количества информации можно записать как:

$$
S_j = \sum_{i=1}^{J} \frac {N_i} {N} S_i
$$

где $J$ - число групп после разбиения.

В качестве критерия выбора признака разбиения можно ввести понятие прироста информации (information gain, IG):

$$
IG_j = S - S_j = S - \sum_{i=1}^{J} \frac {N_i} {N} S_i
$$

Тогда выбор лучшего признака для разбиения запишется через следующее выражение:

$$
Q^{*} = argmax_j \left( IG_j \right) = argmax_j \left( S - \sum_{i=1}^{J} \frac {N_i} {N} S_i \right)
$$

В основе алгоритма построения деревьев решений ID3 лежит принцип жадной максимизации критерия выбора признака. На каждом шаге разбиения выбирается тот признак, при разделении по которому прирост информации оказывается наибольшим. 

Разбиение выборки (построение дерева) продолжается рекурсивно до выполнения критерия останова. В базовом случае этим критерием является достижение энтропии в узле дерева значения $0$, что означает нахождение в узле объектов одного класса, или невозможность уменьшить энтропию в узле никаким разбиением.

#### Использование вещественных признаков

В алгоритме [C4.5](https://en.wikipedia.org/wiki/C4.5_algorithm) добавляется возможность работы с вещественными признаками по следующей схеме. Рассмотрим вещественный признак с индексом $j$. Определим пороги разбиения выборки по этому признаку как упорядоченные уникальные значения этого признака на выборке:

$$
x_j \in \left \{ t_1, t_2, \dots, t_k \right \}_{}
$$

Тогда, выбрав параметр $\theta$, выборку можно разделить по предикату $x_j \le t_m$:

$$
\theta = \left( j, t_m \right )
$$

$$
X_L(\theta) = (X, y) \mid x_j \le t_m
$$

$$
X_R(\theta) = (X, y) \mid x_j > t_m
$$

И выбор лучшего признака и его порога разбиения будет выглядеть как:

$$
\theta^{*} = argmax_\theta (IG_\theta) = argmax_\theta \left( S - \dfrac {N_L} {N} S_L - \dfrac {N_R} {N} S_R \right)
$$

#### Использование различных критериев разбиения

Итак, мы задали функционал для построения деревьев решений на выборках с категориальными и вещественными признаками, который нужно максимизировать на каждом этапе разбиения выборки:

$$
Q \left( X, j, t \right) = F(X) - \dfrac {\mid X_L \mid} {\mid X \mid} F(X_L) - \dfrac {\mid X_R \mid} {\mid X \mid} F(X_R)
$$

где критерий качества разбиения $F(X)$ нами был определен как энтропия Шеннона S(X):

$$
F(X) = S(X) = - \sum_{k=1}^{K} p_k log_2 p_k
$$

В задачах классификации часто используется и другой критерий - [неопределенность Джини](https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity). Он определяется следующим образом:

$$
F(X) = 1 - \sum_{k=1}^{K} p_k^2
$$

Неопределенность Джини можно трактовать как вероятность ошибки классификации объекта, если классифицировать его случайно в соответствии с распределением классов в узле. Является мерой однородности от $0$ (однородной) до $1$ (гетерогенной).

#### Задача регрессии

При решении задачи регрессии идея построения дерева остается той же, но используются другие критерии качества разбиения. Они способствует выбору таких параметров разбиения, при которых значения признаков в подвыборках примерно равны.

Дисперсия ответов вокруг среднего:

$$
F(X) = D(X) = \dfrac {1}{N} \sum_{i=1}^{N} \left( y_i - \dfrac {1}{N} \sum_{j=1}^{N} y_j \right) ^ 2
$$

Среднее отклонение ответов от медианы:

$$
F(X) = \dfrac {1}{N} \sum_{i=1}^{N} \mid y_i - med(y) \mid
$$

#### Ответы в листах

В качестве ответов в листьях дерева будут приниматься: наиболее многочисленный класс у объектов в листе - для задачи классификации, среднее значение ответов по объектам листа - для задачи регрессии:

$$
prediction_C(X) = argmax_k (p_k) = argmax_k \left( \dfrac {N_k}{N} \right)
$$

$$
prediction_R(X) = \dfrac {1}{N} \sum_{i=1}^{N} y_i
$$

#### Алгоритм CART

Алгоритм построения деревьев решений CART похож на C4.5, но имеются отличия. Он поддреживает задачу регрессии по вышеописанному механизму и строит бинарные деревья, т.е. все разбиения в узлах выполняются по предикату: 

$$
\theta(j, t_m): x_j \le t_m
$$

### Используемые источники

1. Энтропия и деревья принятия решений https://habr.com/post/171759/
2. Открытый курс машинного обучения. Тема 3. Классификация, деревья решений и метод ближайших соседей https://habr.com/company/ods/blog/322534/
3. Scikit-learn Decision Trees http://scikit-learn.org/stable/modules/tree.html